In [1]:
%pip install SoccerNet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.3 MB/s eta 0:00:00


In [2]:
import os
import zipfile
from SoccerNet.Downloader import SoccerNetDownloader as SNdl

# Set up the downloader
local_directory = "path/to/SoccerNet"
mySNdl = SNdl(LocalDirectory=local_directory)

# Download the data
mySNdl.downloadDataTask(task="mvfouls", split=["train", "valid", "test", "challenge"], password="s0cc3rn3t")

# Unzip the downloaded files
task_directory = os.path.join(local_directory, "mvfouls")
for split in ["train", "valid", "test", "challenge"]:
    zip_file = os.path.join(task_directory, f"{split}.zip")
    if os.path.exists(zip_file):
        # Create a new folder with the same name as the zip file
        extract_folder = os.path.join(task_directory, split)
        os.makedirs(extract_folder, exist_ok=True)

        # Extract the contents to the new folder
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
        print(f"Extracted {split}.zip to {extract_folder}")
    else:
        print(f"{split}.zip not found")

# Optionally, remove the zip files after extraction
for split in ["train", "valid", "test", "challenge"]:
    zip_file = os.path.join(task_directory, f"{split}.zip")
    if os.path.exists(zip_file):
        os.remove(zip_file)
        print(f"Removed {split}.zip")

Extracted train.zip to path/to/SoccerNet/mvfouls/train
Extracted valid.zip to path/to/SoccerNet/mvfouls/valid
Extracted test.zip to path/to/SoccerNet/mvfouls/test
Extracted challenge.zip to path/to/SoccerNet/mvfouls/challenge
Removed train.zip
Removed valid.zip
Removed test.zip
Removed challenge.zip


In [1]:
# Class name to label index

EVENT_DICTIONARY_action_class = {"Tackling":0,"Standing tackling":1,"High leg":2,"Holding":3,"Pushing":4,
                        "Elbowing":5, "Challenge":6, "Dive":7, "Dont know":8}

INVERSE_EVENT_DICTIONARY_action_class = {0:"Tackling", 1:"Standing tackling", 2:"High leg", 3:"Holding", 4:"Pushing",
                        5:"Elbowing", 6:"Challenge", 7:"Dive", 8:"Dont know"}


EVENT_DICTIONARY_offence_severity_class = {"No offence":0,"Offence + No card":1,"Offence + Yellow card":2,"Offence + Red card":3}

INVERSE_EVENT_DICTIONARY_offence_severity_class = {0:"No offence", 1:"Offence + No card", 2:"Offence + Yellow card", 3:"Offence + Red card"}


EVENT_DICTIONARY_offence_class = {"Offence":0,"Between":1,"No Offence":2, "No offence":2}

INVERSE_EVENT_DICTIONARY_offence_class = {0:"Offence", 1:"Between", 2:"No Offence"}


EVENT_DICTIONARY_severity_class = {"1.0":0,"2.0":1,"3.0":2,"4.0":3,"5.0":4}

INVERSE_EVENT_DICTIONARY_severity_class = {0:"No card", 1:"Borderline No/Yellow", 2:"Yellow card", 3:"Borderline Yellow/Red", 4:"Red card"}


EVENT_DICTIONARY_bodypart_class = {"Upper body":0,"Under body":1}

INVERSE_EVENT_DICTIONARY_bodypart_class = {0:"Upper body", 1:"Under body"}



EVENT_DICTIONARY = {'action_class':EVENT_DICTIONARY_action_class, 'offence_class': EVENT_DICTIONARY_offence_class,
            'severity_class': EVENT_DICTIONARY_severity_class, 'bodypart_class': EVENT_DICTIONARY_bodypart_class, 'offence_severity_class': EVENT_DICTIONARY_offence_severity_class}
INVERSE_EVENT_DICTIONARY = {'action_class':INVERSE_EVENT_DICTIONARY_action_class, 'offence_class': INVERSE_EVENT_DICTIONARY_offence_class,
            'severity_class': INVERSE_EVENT_DICTIONARY_severity_class, 'bodypart_class': INVERSE_EVENT_DICTIONARY_bodypart_class, 'offence_severity_class': INVERSE_EVENT_DICTIONARY_offence_severity_class}

In [18]:
import os
import torch
import json
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#DATA_PATH = 'path/to/SoccerNet/mvfouls'

# Set the desired frame count
DESIRED_FRAME_COUNT = 126

# Load the EVENT_DICTIONARY for mapping annotation labels
EVENT_DICTIONARY = {
    'action_class': {"Tackling": 0, "Standing tackling": 1, "High leg": 2, "Holding": 3, "Pushing": 4,
                     "Elbowing": 5, "Challenge": 6, "Dive": 7, "Dont know": 8},
    'offence_class': {"Offence": 0, "Between": 1, "No Offence": 2, "No offence": 2},
    'severity_class': {"1.0": 0, "2.0": 1, "3.0": 2, "4.0": 3, "5.0": 4},
    'bodypart_class': {"Upper body": 0, "Under body": 1},
    'offence_severity_class': {"No offence": 0, "Offence + No card": 1, "Offence + Yellow card": 2, "Offence + Red card": 3}
}

# Transformation for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def load_filtered_clips_and_labels(DATA_PATH,split, max_samples):
    clips, labels_action, labels_offence, labels_severity, labels_bodypart, labels_offence_severity = [], [], [], [], [], []

    annotations_path = os.path.join(DATA_PATH, split, "annotations.json")
    print(f"Loading annotations from: {annotations_path}")

    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
    print(f"Total actions found in annotations: {len(annotations['Actions'])}")

    offence_count, no_offence_count, skipped_actions = 0, 0, 0
    max_samples = max_samples  # Maximum samples for each class

    for action_index, (action_key, action_data) in enumerate(annotations['Actions'].items()):
        offence_class = action_data['Offence']

        # Filter only 50 samples for each class
        if (offence_class == "Offence" and offence_count >= max_samples) or \
           (offence_class in ["No offence", "No Offence"] and no_offence_count >= max_samples):
            continue

        action_class = action_data['Action class']
        severity_class = action_data['Severity'] if action_data['Severity'].replace('.', '').isdigit() else '1.0'
        bodypart_class = action_data.get('Bodypart', 'Upper body')
        offence_severity = f"{offence_class} + {EVENT_DICTIONARY['severity_class'].get(severity_class, 'No card')}"

        action_label = EVENT_DICTIONARY['action_class'].get(action_class)
        offence_label = EVENT_DICTIONARY['offence_class'].get(offence_class)
        severity_label = EVENT_DICTIONARY['severity_class'].get(severity_class)
        bodypart_label = EVENT_DICTIONARY['bodypart_class'].get(bodypart_class)
        offence_severity_label = EVENT_DICTIONARY['offence_severity_class'].get(offence_severity, 0)

        if None in [action_label, offence_label, severity_label, bodypart_label, offence_severity_label]:
            skipped_actions += 1
            continue

        action_folder = os.path.join(DATA_PATH, split, f"action_{action_key}")

        if not os.path.exists(action_folder):
            skipped_actions += 1
            continue

        action_clips = []
        for clip_idx in range(2):
            clip_path = os.path.join(action_folder, f"clip_{clip_idx}.mp4")
            if not os.path.exists(clip_path):
                continue

            cap = cv2.VideoCapture(clip_path)
            frames = []
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
                frame = transform(frame)
                frames.append(frame)
            cap.release()

            # Sample or pad frames to ensure uniform frame count
            if len(frames) > DESIRED_FRAME_COUNT:
                indices = np.linspace(0, len(frames) - 1, DESIRED_FRAME_COUNT).astype(int)
                frames = [frames[i] for i in indices]
            elif len(frames) < DESIRED_FRAME_COUNT:
                frames += [frames[-1]] * (DESIRED_FRAME_COUNT - len(frames))

            video_tensor = torch.stack(frames, dim=0)
            action_clips.append(video_tensor)

            # Print tensor shape and size for debugging
            print(f"Action {action_key}, Clip {clip_idx} tensor shape: {video_tensor.shape}")
            tensor_size = video_tensor.element_size() * video_tensor.nelement() / (1024**2)
            print(f"Tensor size: {tensor_size:.2f} MB")

        if action_clips:
            clips.append(action_clips)
            labels_action.append(action_label)
            labels_offence.append(offence_label)
            labels_severity.append(severity_label)
            labels_bodypart.append(bodypart_label)
            labels_offence_severity.append(offence_severity_label)

            if offence_class == "Offence":
                offence_count += 1
            else:
                no_offence_count += 1

            print(f"Added action {action_key} with {len(action_clips)} clips to dataset.")

        # Stop if we have 50 samples for each class
        if offence_count >= max_samples and no_offence_count >= max_samples:
            break

    print("\nSummary:")
    print(f"Total actions loaded: {len(clips)}")
    print(f"Total actions skipped: {skipped_actions}")
    if clips:
        print(f"Example clip shape: {clips[0][0].shape}")
        print(f"First action label: {labels_action[0]}")

    return clips, labels_action, labels_offence, labels_severity, labels_bodypart, labels_offence_severity


# # Load datasets for each split
# datasets = {}
# for split in ['train', 'valid', 'test']:
#     clips, labels_action, labels_offence, labels_severity, labels_bodypart, labels_offence_severity = load_filtered_clips_and_labels(DATA_PATH,split)
#     datasets[split] = {
#         'clips': clips,
#         'labels': {
#             'action': labels_action,
#             'offence': labels_offence,
#             'severity': labels_severity,
#             'bodypart': labels_bodypart,
#             'offence_severity': labels_offence_severity
#         }
#     }

# # Display dataset info for verification
# for split in ['train', 'valid', 'test']:
#     print(f"{split.capitalize()} set:")
#     print("Number of samples:", len(datasets[split]['clips']))
#     print("Example label structure:", datasets[split]['labels']['action'][0] if datasets[split]['labels']['action'] else "No labels")


In [19]:
import torch
import torch.nn as nn
import torchvision.models as models

class MultiTaskActionRecognitionModel(nn.Module):
    def __init__(self, num_classes_action=9, num_classes_offence=3, num_classes_severity=5,
                 num_classes_bodypart=2, num_classes_offence_severity=4, freeze_backbone=True):
        super(MultiTaskActionRecognitionModel, self).__init__()

        # Load pretrained ResNet-18 3D backbone
        r3d_18 = models.video.r3d_18(pretrained=True)

        # Optionally freeze backbone layers
        if freeze_backbone:
            for param in r3d_18.parameters():
                param.requires_grad = False

        # Remove the last fully connected layer
        self.backbone = nn.Sequential(*list(r3d_18.children())[:-1])  # Output shape will be [batch, 512, 1, 1, 1]

        # Define custom fully connected layers for each task
        self.fc_action = nn.Linear(512, num_classes_action)
        self.fc_offence = nn.Linear(512, num_classes_offence)
        self.fc_severity = nn.Linear(512, num_classes_severity)
        self.fc_bodypart = nn.Linear(512, num_classes_bodypart)
        self.fc_offence_severity = nn.Linear(512, num_classes_offence_severity)

    def forward(self, x):
        # Extract features using the backbone
        x = self.backbone(x)
        x = x.view(x.size(0), -1)  # Flatten to [batch, 512]

        # Forward through task-specific fully connected layers
        action_out = self.fc_action(x)
        offence_out = self.fc_offence(x)
        severity_out = self.fc_severity(x)
        bodypart_out = self.fc_bodypart(x)
        offence_severity_out = self.fc_offence_severity(x)

        return action_out, offence_out, severity_out, bodypart_out, offence_severity_out


In [20]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import GradScaler, autocast
#from model import MultiTaskActionRecognitionModel
#from data import load_filtered_clips_and_labels
from torchvision import transforms
from PIL import Image
torch.cuda.empty_cache()
import gc
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define parameters
batch_size = 2
num_epochs = 10
learning_rate = 0.001
max_samples = 15
fixed_frame_count = 126  # Standardized frame count for all clips
DATA_PATH = 'mvfouls'

# Transformation for preprocessing
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class ActionDataset(Dataset):
    def __init__(self, clips, labels_action, labels_offence, labels_severity, labels_bodypart, labels_offence_severity):
        self.clips = clips
        self.labels_action = labels_action
        self.labels_offence = labels_offence
        self.labels_severity = labels_severity
        self.labels_bodypart = labels_bodypart
        self.labels_offence_severity = labels_offence_severity

    def __len__(self):
        return len(self.clips)

    def pad_or_truncate(self, clip):
        # Pad or truncate each clip to fixed frame count
        frames = []
        for i, frame in enumerate(clip):
            if not isinstance(frame, torch.Tensor):  # Ensure the frame is not already a tensor
                frame = transform(frame)  # Apply transforms
            frames.append(frame)
            if len(frames) == fixed_frame_count:  # Truncate
                break
        # Pad if necessary
        while len(frames) < fixed_frame_count:
            frames.append(torch.zeros_like(frames[0]))  # Padding with empty frames
        return torch.stack(frames)

    def __getitem__(self, idx):
        clip = self.clips[idx]
        padded_clip = self.pad_or_truncate(clip)
        return (padded_clip,
                self.labels_action[idx],
                self.labels_offence[idx],
                self.labels_severity[idx],
                self.labels_bodypart[idx],
                self.labels_offence_severity[idx])

# Load filtered dataset
train_clips, train_labels_action, train_labels_offence, train_labels_severity, train_labels_bodypart, train_labels_offence_severity = load_filtered_clips_and_labels(DATA_PATH, "train", max_samples=max_samples)
valid_clips, valid_labels_action, valid_labels_offence, valid_labels_severity, valid_labels_bodypart, valid_labels_offence_severity = load_filtered_clips_and_labels(DATA_PATH,"valid", max_samples=max_samples)

# Create Datasets and DataLoaders
train_dataset = ActionDataset(train_clips, train_labels_action, train_labels_offence, train_labels_severity, train_labels_bodypart, train_labels_offence_severity)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = ActionDataset(valid_clips, valid_labels_action, valid_labels_offence, valid_labels_severity, valid_labels_bodypart, valid_labels_offence_severity)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
print('Data loaded')

# Reshape the inputs to [batch_size, frames_clip_0 * frames_clip_1, channels, height, width]
def reshape_tensor(x):
    b, f0, f1, c, h, w = x.shape
    return x.view(b, f0*f1, c, h, w).permute(0, 2, 1, 3, 4)

# Initialize model, loss, and optimizer
model = MultiTaskActionRecognitionModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

accumulation_steps = 4  # Number of steps to accumulate gradients

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    optimizer.zero_grad()  # Reset gradients at the beginning of each epoch

    for i, (inputs, action_labels, offence_labels, severity_labels, bodypart_labels, offence_severity_labels) in enumerate(train_loader):
        inputs = reshape_tensor(inputs).to(device)

        # Move labels to device
        action_labels = action_labels.to(device)
        offence_labels = offence_labels.to(device)
        severity_labels = severity_labels.to(device)
        bodypart_labels = bodypart_labels.to(device)
        offence_severity_labels = offence_severity_labels.to(device)

        # Forward pass with mixed precision
        action_out, offence_out, severity_out, bodypart_out, offence_severity_out = model(inputs)
        loss = (criterion(action_out, action_labels) +
                criterion(offence_out, offence_labels) +
                criterion(severity_out, severity_labels) +
                criterion(bodypart_out, bodypart_labels) +
                criterion(offence_severity_out, offence_severity_labels))


        # Update gradients only after accumulation steps
        optimizer.zero_grad()  # Reset gradients for the next accumulation
        train_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss / len(train_loader):.4f}")
    # Validation loop
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():  # Disable gradient tracking for validation
        for inputs, action_labels, offence_labels, severity_labels, bodypart_labels, offence_severity_labels in valid_loader:
            # Check the shape of inputs
            #[batch_size, frames_clip_0, frames_clip_1, channels, height, width]
            print(f"Input shape before adjustment: {inputs.shape}")  # Should print [2, 126, 126, 3, 64, 64] per action

            inputs = reshape_tensor(inputs)


            inputs = inputs.to(device)
            action_labels = action_labels.to(device)
            offence_labels = offence_labels.to(device)
            severity_labels = severity_labels.to(device)
            bodypart_labels = bodypart_labels.to(device)
            offence_severity_labels = offence_severity_labels.to(device)

            action_out, offence_out, severity_out, bodypart_out, offence_severity_out = model(inputs)

            # Calculate validation loss
            loss_action = criterion(action_out, action_labels)
            loss_offence = criterion(offence_out, offence_labels)
            loss_severity = criterion(severity_out, severity_labels)
            loss_bodypart = criterion(bodypart_out, bodypart_labels)
            loss_offence_severity = criterion(offence_severity_out, offence_severity_labels)

            loss = loss_action + loss_offence + loss_severity + loss_bodypart + loss_offence_severity
            valid_loss += loss.item()

    print(f"Validation Loss: {valid_loss / len(valid_loader):.4f}")
    
    # After each training epoch
    gc.collect()
    torch.cuda.empty_cache()

# Save the model
os.makedirs("saved_models", exist_ok=True)
torch.save(model.state_dict(), "saved_models/action_recognition_model.pth")
print("Model saved!")


Loading annotations from: mvfouls\train\annotations.json
Total actions found in annotations: 2916
Action 0, Clip 0 tensor shape: torch.Size([126, 3, 32, 32])
Tensor size: 1.48 MB
Action 0, Clip 1 tensor shape: torch.Size([126, 3, 32, 32])
Tensor size: 1.48 MB
Added action 0 with 2 clips to dataset.
Action 1, Clip 0 tensor shape: torch.Size([126, 3, 32, 32])
Tensor size: 1.48 MB
Action 1, Clip 1 tensor shape: torch.Size([126, 3, 32, 32])
Tensor size: 1.48 MB
Added action 1 with 2 clips to dataset.
Action 2, Clip 0 tensor shape: torch.Size([126, 3, 32, 32])
Tensor size: 1.48 MB
Action 2, Clip 1 tensor shape: torch.Size([126, 3, 32, 32])
Tensor size: 1.48 MB
Added action 2 with 2 clips to dataset.
Action 3, Clip 0 tensor shape: torch.Size([126, 3, 32, 32])
Tensor size: 1.48 MB
Action 3, Clip 1 tensor shape: torch.Size([126, 3, 32, 32])
Tensor size: 1.48 MB
Added action 3 with 2 clips to dataset.
Action 4, Clip 0 tensor shape: torch.Size([126, 3, 32, 32])
Tensor size: 1.48 MB
Action 4, Cli

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.94 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 6.70 GiB is allocated by PyTorch, and 28.33 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)